# Simple Usage

The main functionality of the package is accessible through `nbed.nbed()`.

There are three ways to provide arguments to the function:
1. passing a path to a config `.json` file.  
2. passing named arguments directly.
3. passing an NbedConfig model.

Note that named arguments which are explicitly added will overwrite the config input from a file or model.

## Example Config file

First lets see what's in the file before we pass it to the main function.

In [1]:
import json
with open("test_config.json") as f:
    config_from_file = json.load(f)
    
config_from_file

{'geometry': '3\n\nO   0.0000  0.000  0.115\nH   0.0000  0.754  -0.459\nH   0.0000  -0.754  -0.459',
 'n_active_atoms': 1,
 'basis': 'STO-3G',
 'xc_functional': 'b3lyp',
 'projector': 'mu',
 'localization': 'spade',
 'convergence': 1e-06,
 'charge': 0,
 'spin': 0,
 'unit': 'angstrom',
 'symmetry': False,
 'mu_level_shift': 1000000.0,
 'run_ccsd_emb': True,
 'run_fci_emb': True,
 'run_virtual_localization': True,
 'run_dft_in_dft': True,
 'n_mo_overwrite': [None, None],
 'max_ram_memory': 4000,
 'occupied_threshold': 0.95,
 'virtual_threshold': 0.95,
 'max_shells': 4,
 'init_huzinaga_rhf_with_mu': False,
 'max_hf_cycles': 50,
 'max_dft_cycles': 50,
 'force_unrestricted': False,
 'mm_coords': None,
 'mm_charges': None,
 'mm_radii': None}

In [2]:
from nbed import nbed

result = nbed(config="test_config.json")

/Users/michaelwilliamsdelabastida/Code/Nbed/.venv/lib/python3.13/site-packages/pyscf/dft/libxc.py:512: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


## Adding arguments directly

In [3]:
geometry= "3\n\nO   0.0000  0.000  0.115\nH   0.0000  0.754  -0.459\nH   0.0000  -0.754  -0.459"

result = nbed(geometry=geometry, n_active_atoms=2, basis="sto-3g", xc_functional="b3lyp", projector="mu", localization="spade",convergence=1e-6, charge=0, spin=0)

### Overwriting arguments
Let's now overwrite some arguments, using the same config to embed some atoms of methane.

In [4]:
from nbed import nbed

methane = """5

C\t0.0\t0.0\t0.0
H\t0.5288\t0.1610\t0.9359
H\t0.2051\t0.8240\t-0.6786
H\t0.3345\t-0.9314\t-0.4496
H\t-1.0685\t-0.0537\t0.1921
#     """

result = nbed(config="test_config.json", geometry=methane)

## Using an NbedConfig model.

The final option is to directly pass the pydantic model that Nbed uses internally to validate data.

In [5]:
from nbed.config import NbedConfig

config = NbedConfig(geometry=geometry, n_active_atoms=2, basis="sto-3g", xc_functional="b3lyp", projector="mu", localization="spade",convergence=1e-6, charge=0, spin=0)
result = nbed(config)

In [6]:
config = NbedConfig(**config_from_file)
result = nbed(config)

# Command-line Interface

It is also possible to run nbed from the command line, the `nbed` command will be installed with the package and allows you to input the path to a config file.

This can be useful for running nbed over ssh.

# Results

Results for the `mu` and `huzinaga` projectors are stored separately, let's take a look at what's included in there.

In [25]:
result.mu.keys()

dict_keys(['scf', 'v_emb', 'mo_energies_emb_pre_del', 'mo_energies_emb_post_del', 'correction', 'beta_correction', 'cl', 'e_rhf', 'classical_energy', 'e_ccsd', 'ccsd_emb', 'e_fci', 'fci_emb', 'hf_emb', 'scf_dft', 'v_emb_dft', 'dft_correction', 'dft_correction_beta', 'e_dft_in_dft', 'emb_dft', 'second_quantised'])

## PySCf Object

If you want to contnue to use PySCF methods on the embedded system, you can get the embedded PySCF object `result.mu["embedded_scf"]` (usually a UKS object), together with a correction to the energy which represents the environment `result.mu["classical_energy"]`. 

In [31]:
result.mu["scf"]

In [30]:
result.mu["classical_energy"]

np.float64(-14.229086664077219)

## Second Quantised Hamiltonian

The second quantised electronic structure hamiltonian will be the main thing you need if you're planning to run a quantum algorithm. You'll need to pair this with a [Fermion-Qubit encoding](https://ferrmion.readthedocs.io/) to create a qubit Hamiltonian that's well optimised to the device you intend to use.

Nbed used the spin-orbit format, where the two spins of molecular orbital $i$: $(i_{\uparrow}, i_{\downarrow})$ map to indices $(2i, 2i+1)$

In [26]:
constant, one_e_terms, two_e_terms = result.mu["second_quantised"]

In [27]:
constant, one_e_terms.shape, two_e_terms.shape

(np.float64(-14.229086664077219), (10, 10), (10, 10, 10, 10))

##  Other information

Most of the relevant information created or used in the embedding is accessible:

In [41]:
for k, v in result.mu.items():
    print(f"{k}: {type(v)}")

scf: <class 'pyscf.scf.uhf.UHF'>
v_emb: <class 'numpy.ndarray'>
mo_energies_emb_pre_del: <class 'numpy.ndarray'>
mo_energies_emb_post_del: <class 'numpy.ndarray'>
correction: <class 'numpy.float64'>
beta_correction: <class 'numpy.float64'>
cl: <class 'nbed.localizers.virtual.concentric.ConcentricLocalizer'>
e_rhf: <class 'numpy.float64'>
classical_energy: <class 'numpy.float64'>
e_ccsd: <class 'numpy.float64'>
ccsd_emb: <class 'numpy.float64'>
e_fci: <class 'numpy.float64'>
fci_emb: <class 'numpy.float64'>
hf_emb: <class 'numpy.float64'>
scf_dft: <class 'pyscf.dft.uks.UKS'>
v_emb_dft: <class 'numpy.ndarray'>
dft_correction: <class 'numpy.float64'>
dft_correction_beta: <class 'numpy.float64'>
e_dft_in_dft: <class 'numpy.float64'>
emb_dft: <class 'numpy.float64'>
second_quantised: <class 'tuple'>
